In [7]:
# SPARK_HOME="/Users/ssv/spark-2.2.0-bin-hadoop2.7"
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='python3'
# os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["SPARK_HOME"]='/Users/ssv/spark-2.2.0-bin-hadoop2.7'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
os.environ["PYSPARK_PYTHON"] = 'python3'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.0
      /_/

Using Python version 3.6.2 (default, Sep 21 2017 18:29:43)
SparkSession available as 'spark'.


In [8]:
filename = '/Users/ssv/newprolab/labs/lab07/DO_record_per_line.json'

In [409]:
df = spark.read.json(filename)

In [628]:
#df_ru = df.filter(df.lang == 'ru')
df_ru = df.filter(df.lang == 'es')

In [629]:
df_ru.show(2)

+--------------------+--------------------+---+----+--------------------+--------+
|                 cat|                desc| id|lang|                name|provider|
+--------------------+--------------------+---+----+--------------------+--------+
|                    |A través de difer...| 59|  es|El ABC del empren...|Coursera|
|2/biology_life_sc...|Aprenderemos cómo...|124|  es|Pensamiento Cient...|Coursera|
+--------------------+--------------------+---+----+--------------------+--------+
only showing top 2 rows



In [630]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, StopWordsRemover

In [631]:
regexTokenizer = RegexTokenizer(inputCol="desc", outputCol="words", minTokenLength=3, \
                                                 gaps=True, pattern='((?U)[^\w]|[+])')
r_tokenize_df_ru = regexTokenizer.transform(df_ru)

In [ ]:
# # “danish”, “dutch”, “english”, “finnish”, “french”, “german”, “hungarian”, “italian”, 
# # “norwegian”, “portuguese”, “russian”, “spanish”, “swedish” and “turkish”

# remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
# remover.loadDefaultStopWords('russian')
# remover.transform(tokenize_df_ru).show(truncate=False)


In [632]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)
featurizedData = hashingTF.transform(r_tokenize_df_ru)

In [633]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [634]:
rescaledData.count()

1374

In [635]:
from pyspark.ml.feature import Normalizer
normalizer_l2 = Normalizer(inputCol="features", outputCol="normalized_features", p=2)

normalized_rescaledData = normalizer_l2.transform(rescaledData)

In [636]:
normalized_rescaledData.printSchema()

root
 |-- cat: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- id: long (nullable = true)
 |-- lang: string (nullable = true)
 |-- name: string (nullable = true)
 |-- provider: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rawFeatures: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- normalized_features: vector (nullable = true)



In [527]:
def calc_cosine_similarity(a, b):
    return a.dot(b)

In [648]:
#cd = normalized_rescaledData.filter('id = 16704').select('normalized_features').take(1)
#cd = normalized_rescaledData.filter('id = 13702').select('normalized_features').take(1)
cd = normalized_rescaledData.filter('id = 11556').select('normalized_features').take(1)
cd



[Row(normalized_features=SparseVector(10000, {249: 0.0908, 522: 0.0685, 527: 0.058, 643: 0.0698, 709: 0.0948, 808: 0.0024, 841: 0.0711, 909: 0.0607, 993: 0.0828, 1047: 0.0601, 1072: 0.079, 1096: 0.0456, 1203: 0.0044, 1236: 0.0784, 1312: 0.0079, 1448: 0.0379, 1606: 0.0015, 1930: 0.0526, 1959: 0.0078, 2080: 0.008, 2090: 0.0504, 2171: 0.0771, 2249: 0.0504, 2369: 0.1091, 2384: 0.0771, 2460: 0.0069, 2726: 0.0414, 2739: 0.098, 2850: 0.0315, 2906: 0.05, 2922: 0.1266, 3084: 0.0539, 3138: 0.0078, 3270: 0.5431, 3375: 0.0771, 3429: 0.0354, 3586: 0.0053, 3855: 0.0077, 3921: 0.0077, 4213: 0.1175, 4306: 0.0948, 4423: 0.0537, 4723: 0.0644, 4758: 0.1462, 5034: 0.026, 5260: 0.0634, 5262: 0.0041, 5300: 0.0609, 5314: 0.0358, 5373: 0.0075, 5529: 0.0998, 5612: 0.0081, 5641: 0.0908, 5660: 0.0644, 5710: 0.0145, 5772: 0.0741, 5786: 0.0078, 5798: 0.0472, 5820: 0.0885, 5872: 0.0529, 5990: 0.1091, 6105: 0.4655, 6119: 0.1709, 6215: 0.0283, 6256: 0.0681, 6276: 0.1017, 6306: 0.0805, 6316: 0.1123, 6324: 0.0083, 6443

In [649]:
choosen_doc = cd[0].normalized_features

In [651]:
fdf = normalized_rescaledData.select(normalized_rescaledData.id, 
                                     normalized_rescaledData.normalized_features.alias('nf'),
                                     normalized_rescaledData.name)

In [653]:
rdd = fdf.rdd.map(lambda x: (x[0], float(x[1].dot(choosen_doc)), x[2]))

In [654]:
cos_df = spark.createDataFrame(rdd, schema=['id','cm', 'name'])

In [657]:
cos_df.orderBy(['cm', 'name', 'id'], ascending=[0, 1, 1]).show(11)

+-----+-------------------+--------------------+
|   id|                 cm|                name|
+-----+-------------------+--------------------+
|11556|                1.0|Aprendizaje Colab...|
|10384|0.26708008906411607|Marketing Digital...|
|16488| 0.2520262532288868|Aprendizaje basad...|
|  468|0.22799514721585262|Tecnologías de in...|
|22710| 0.2029379209171897|Aplicaciones crea...|
|13461| 0.1984825271366596|TIC para la enseñ...|
|21707|0.18499702189474398|Tecnologías para ...|
|19330| 0.1822903277838439|Introducción al m...|
|23357|0.18044046069955805|Diseña Cursos Din...|
|10447| 0.1743993046639929|Mejores calificac...|
| 9465|0.15857582159660835|Desarrolla tu esp...|
+-----+-------------------+--------------------+
only showing top 11 rows



In [658]:
res_lst = cos_df.select('id').orderBy(['cm', 'name', 'id'], ascending=[0, 1, 1]).take(11)

courses = []
courses.append(res_lst[0].id)

recommended = []
for i in range(1,11):
    recommended.append(res_lst[i].id)

courses.append(recommended)
print(courses)

[11556, [10384, 16488, 468, 22710, 13461, 21707, 19330, 23357, 10447, 9465]]


en
[23126, [14760, 13665, 13782, 20638, 24419, 15909, 2724, 25782, 17499, 13348]]
[21617, [21609, 21673, 21081, 21616, 19417, 22298, 380, 8110, 116, 16971]]


es
[16627, [11431, 17961, 17964, 5687, 12247, 16694, 5558, 12660, 11575, 9563]]
[11556, [10384, 16488, 468, 22710, 13461, 21707, 19330, 23357, 10447, 9465]]

ru
[16704, [1219, 1327, 20362, 1228, 55, 26980, 1236, 1247, 1365, 913]]
[13702, [864, 21079, 1111, 792, 1410, 8123, 8313, 1041, 1033, 1396]]





In [659]:
result = {
    '23126' : [14760, 13665, 13782, 20638, 24419, 15909, 2724, 25782, 17499, 13348],
    '21617' : [21609, 21673, 21081, 21616, 19417, 22298, 380, 8110, 116, 16971],
    '16627' : [11431, 17961, 17964, 5687, 12247, 16694, 5558, 12660, 11575, 9563],
    '11556' : [10384, 16488, 468, 22710, 13461, 21707, 19330, 23357, 10447, 9465],
    '16704' : [1219, 1327, 20362, 1228, 55, 26980, 1236, 1247, 1365, 913],
    '13702' : [864, 21079, 1111, 792, 1410, 8123, 8313, 1041, 1033, 1396]
        }

In [661]:
import json
with open('lab07s.json', 'w') as file:
    json.dump(result, file)